In [1]:
from pprint import pprint
import re

from datasets import load_dataset, Dataset, DatasetDict
import jsonlines as jsonl

In [2]:
dataset = load_dataset(
    "THUDM/AgentInstruct",
    split="kg+db+mind2web+os+webshop+alfworld",
)

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating alfworld split:   0%|          | 0/336 [00:00<?, ? examples/s]

Generating db split:   0%|          | 0/538 [00:00<?, ? examples/s]

Generating kg split:   0%|          | 0/324 [00:00<?, ? examples/s]

Generating mind2web split:   0%|          | 0/122 [00:00<?, ? examples/s]

Generating os split:   0%|          | 0/195 [00:00<?, ? examples/s]

Generating webshop split:   0%|          | 0/351 [00:00<?, ? examples/s]

In [6]:
make_chatml = lambda name, role, content: dict(
    name=name, role=role, content=content,
)

system = lambda name, content: make_chatml(
    role="system",
    name=name,
    content=content,
)

situation = lambda content: system(name="situation", content=content)
thought = lambda content: system(name="thought", content=content)
information = lambda content: system(name="information", content=content)
me = lambda content, name=None: make_chatml(
    role="assistant",
    content=content,
    name=name,
)

person = lambda content, name=None: make_chatml(
    role="user",
    content=content,
    name=name,
)


In [4]:
dataset[0]

{'conversations': [{'from': 'human',
   'loss': None,
   'value': "You are an agent that answers questions based on the knowledge stored in a knowledge base. To achieve this, you can use the following tools to query the KB.\n\n1. get_relations(variable: var) -> list of relations\nA variable can be either an entity or a set of entities (i.e., the result of a previous query). This function helps to navigate all relations in the KB connected to the variable, so you can decide which relation is the most useful to find the answer to the question.\nA simple use case can be 'get_relations(Barack Obama)', which finds all relations/edges starting from the entity Barack Obama.\nThe argument of get_relations should always be an entity or a variable (e.g., #0) and not anything else.\n\n2. get_neighbors(variable: var, relation: str) -> variable\nGiven a variable, this function returns all entities connected to the variable via the given relation. Note that, get_neighbors() can only be used after ge

In [7]:
def to_chatml(row):
    conversations = row["conversations"]
    instructions = conversations.pop(0)["value"]
    newline = "\n"
    
    system_message = (
        "An AI Assistant is helping a user with a complex multi-step task."
        " The Assistant is given a set of tools including instructions for using them."
        " The Assistant thinks step by step and uses the tools to complete the task."
    )

    # Turn into chatml
    chatml = [
        situation(system_message),
        person(instructions),
    ]

    for turn in conversations:
        from_ = turn["from"]
        value = turn["value"]

        if from_ == "human":
            if value.startswith("Observation:"):
                chatml.append(information(value))
            else:
                chatml.append(person(value))
        else:
            if value.startswith("Thought:"):
                chatml.append(thought(value.replace("Thought: ", "Thought:\n\n")))
            else:
                chatml.append(me(value))

    return dict(chatml=chatml)


In [8]:
dataset = dataset.map(to_chatml)

Map:   0%|          | 0/1866 [00:00<?, ? examples/s]

In [10]:
dataset = dataset.remove_columns(['conversations', 'id'])

In [12]:
dataset = DatasetDict(
    dict(
        train=dataset,
    )
)

In [13]:
dataset.push_to_hub("diwank/agent_instructions-chatml", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]